---

# 230829~
## 목표
1. 목표 함수 성능 향상
2. 파라미터 수정 용이하도록 앞에 배치

| Number of targets | Paper | seed 0 | seed 42 |
|-----|----------|----------|----------|
| 10   |761.25|767.79|774.54|
| 20   |1569.51|1620.20|1468.81|
| 30   |2319.36|2171.93|2339.14|
| 40   |2941.82|2821.07|2957.20|
| 50   |3286.20|3325.39|3124.97|
| 60   |3520.40|3156.24|3265.10|
| 70   |3694.57|3477.84|3747.54|
| 80   |3829.13|3598.11|3722.80|
| 90   |3882.83|3640.43|3714.42|
| 100  |3962.09|3531.34|3742.08|


In [20]:
import numpy as np
# 시드를 고정
np.random.seed(0)

# 발사대를 5대로 가정하여 진행
W = 5
# 위협 표적의 개수를 10개에서 100개까지 10개씩 증가시키며 실행
T = 10
# 표적 t에 할당할 수 있는 유도탄의 개수는 1개로 가정
FN_T = [1] * T

M = sum(FN_T)

# 0에서 4 사이의 랜덤한 값으로 구성된 W*T 행렬 생성 : 최초발사가능시점
ft_wt = np.random.rand(W, T) * 4

# 최소 3초 이상의 간격을 갖도록 설정
min_lt_wt = ft_wt + 3

# 발사가능시점으로부터 최소 3초 이상~ 전체 시간 10초 이내의 간격 행렬 생성
inter_lt_wt = 10 - min_lt_wt

# 3에서 10 사이의 랜덤한 값으로 구성된 W*T 행렬 생성 : 최후발사가능시점
lt_wt = np.random.rand(W, T) * inter_lt_wt + min_lt_wt

# 최초발사가능 시점과 최후발사가능 시점 사이의 간격이 3초 이상을 갖는지 확인
time_intervals_diff = lt_wt - ft_wt

# 0과 99 사이의 실수로 PK_wt 배열 초기화
PK_wt = np.random.uniform(0, 99, (W, T))

# tau 정의 : 발사대 w에서 유도탄 m의 발사 시점 (튜플(발사시점, 유도탄 m)로 구성된 리스트)
# 발사대 만큼의 빈 리스트로 구성
tau = [[] for _ in range(W)]

# M을 W로 나누어 리스트에 적절히 분배
quotient, remainder = divmod(M, W)
m_w = [quotient] * W

# 나머지 값을 리스트의 앞부분에 추가
for i in range(remainder):
    m_w[i] += 1

print(m_w)



[2, 2, 2, 2, 2]


In [21]:
import copy
# Initialize a 3D binary matrix of zeros
theta_wt = np.zeros((W, T, M), dtype=int)

# 미사일이 소모되는 리스트 left_FN_T으로 복사
left_FN_T = copy.copy(FN_T)

# 발사대에 적재된 유도탄 수량 리스트 left_m_w로 복사
left_m_w = copy.copy(m_w)

# 행렬의 원소들을 1차원 배열로 변환
flattened = PK_wt.flatten()

# 원소들을 내림차순으로 정렬
sorted_indices = np.argsort(flattened)[::-1]


i = 0
m = 0
indices_list = []
while True :
    if i == T * W :
        break
    # i번째 요격확률의 인덱스 v
    v = sorted_indices[i]
    
    row = v // T # Weapon
    col = v % T # Target
    print(f"Value: {flattened[v]}, Index: (Weapon : {row}, Target : {col})")
    
    # 해당 Target의 할당할 수 있는 유도탄의 개수가 남아 있다면,
    # 해당 Weapon의 적재된 유도탄이 남아 있다면,
    if left_FN_T[col] > 0 and left_m_w[row] > 0:
        
        

        # 빈 리스트 일 경우 최초 요격 가능 시간 부여
        if len(tau[row]) == 0:
            tau[row].append((ft_wt[row][col], m))
        else : 
            last_launch_time = tau[row][-1][0]
            # 요격 가능 할 경우,
            if last_launch_time+1 <=  lt_wt[row][col] and ft_wt[row][col] <= 10:
                tau[row].append((max(last_launch_time+1.0001, ft_wt[row][col]) , m))

            else :  
                # 할당하지 않고 다음 인덱스로 이동
                i += 1
                continue
            
        
             
        # 할당할 유도탄 개수 감소
        left_FN_T[col] -= 1
        left_m_w[row] -= 1
        # 해당 W T M에 1 부여
        theta_wt[row][col][m] = 1
        # 무기가 할당된 indices_list에 추가
        indices_list.append(v)

        # 다음 미사일 인덱스로 이동
        m += 1

    # 해당 Target의 할당할 수 있는 유도탄의 개수가 남아 있지 않다면,
    elif left_FN_T[col] > 0:
        print(f"not enough Missile on Weapon {row}")
    
    else :
        print(f"not enough Missile for Target {col}")
    
    # 할당할 수 있는 유도탄의 개수를 모두 소모했다면 종료.
    if sum(left_FN_T) == 0:
        break
    

    # 다음으로 요격확률이 높은 인덱스로 이동
    i += 1


Value: 98.88585365021879, Index: (Weapon : 4, Target : 9)
Value: 95.25666596662639, Index: (Weapon : 0, Target : 3)
Value: 94.65227983759917, Index: (Weapon : 2, Target : 2)
Value: 94.32215214018152, Index: (Weapon : 0, Target : 9)
not enough Missile for Target 9
Value: 91.02877876072269, Index: (Weapon : 4, Target : 7)
Value: 88.75811298925524, Index: (Weapon : 4, Target : 0)
not enough Missile on Weapon 4
Value: 88.30041214655154, Index: (Weapon : 4, Target : 3)
not enough Missile for Target 3
Value: 87.29180082363042, Index: (Weapon : 1, Target : 8)
Value: 87.229216514005, Index: (Weapon : 1, Target : 6)
Value: 83.79445857464165, Index: (Weapon : 1, Target : 1)
not enough Missile on Weapon 1
Value: 80.56598415054525, Index: (Weapon : 1, Target : 4)
not enough Missile on Weapon 1
Value: 79.8132049155625, Index: (Weapon : 4, Target : 4)
not enough Missile on Weapon 4
Value: 72.7842081901369, Index: (Weapon : 0, Target : 2)
not enough Missile for Target 2
Value: 71.8001737021444, Index

In [22]:
"""
목표함수 (1)
해당 목적함수는 결정변수 theta_wt와 요격확률의 곱의 합을 최대화함으로써 요격확률이 높을 때 표적-발사대 쌍의 발사 시점에 무기할당을 하는 의미로 해석할 수 있다.
"""
def objective_fun_1(theta_wt):
    res = 0
    for t in range(T):
        
        a = 1
        for w in range(W):
            for m in range(M):
                p = 1 - PK_wt[w,t] * theta_wt[w,t,m]
                a *= p
        
        # print(f"타깃 {t}에 대한 성공 값 : {1-a}")
        res += 1-a
    return res

"""
제약조건 (2)
수식 (2)는 동일 유도탄이 다수의 표적에 중복하여 할당되지 않기 위한 제약 조건이다.
"""

# 동일 유도탄이 다수의 표적에 중복 할당 X
def constraint2(theta_wt):
    W, T, M = theta_wt.shape
    for m in range(M):
        for w in range(W):
            # If the sum exceeds 1, immediately return False
            if np.sum(theta_wt[w,:,m]) > 1:
                return False
    # If none of the sums exceed 1, return True
    return True

"""
제약조건 (3)
각 위협 표적에 할당할 수 있는 유도탄의 개수를 제한하기 위하여 수식 (3)과 같은 제약식을 적용한다.
"""
# constraint3
def constraint3(theta_wt):
    W, T, M = theta_wt.shape
    for t in range(T):
        if np.sum(theta_wt[:,t,:]) > FN_T[t]:
            return False
    return True
"""
제약조건 (4)
한 발사대가 연속발사를 하기 위해서는 발사절차 및 발사대 안정화를 위한 시간을 고려하여야 하므로, 수식 (4)과 같이 한 발사대 상의 연속적 발사를 위한 최소 발사 간격을 제약으로 고려한다.
"""

def constraint4(tau):
    for t in tau:
        for i in range(len(t)-1):
            if t[i+1][0]-t[i][0] < 1 :
                print (t[i+1][0]-t[i][0]) # 0.9999999999999996 으로 나와서
                return False
        
    return True
"""
제약조건 (5)
각 발사대에서 발사된 유도탄은 제한된 공간 영역 상에서 표적을 요격할 수 있다.
표적의 예상 궤적과 발사대 위치에 따른 요격 공간으로부터 교전 가능한 유도탄 발사 시간이 계산되어진다.
위협 표적에 대해 서 교전 가능한 발사 시간 구간 내에서 발사 시점이 산출되어야만 교전 가능하므로, 수식 (5)와 같이 표적 t에 대한 발사대 w에서 발사 가능한 시간에 대한 제약 조건을 고려해야 한다.
"""
def constraint5(tau, theta_wt):
    W, T, M = theta_wt.shape
    for m in range(M):
        indices = np.where(theta_wt[:,:,m] == 1)
        if len(indices[0]) == 0 :
            continue

        w = int(indices[0])
        t = int(indices[1])
        
        ft = ft_wt[w][t]
        lt = lt_wt[w][t]

        tau_wm_m_filtered = list(filter(lambda x: x[1] == m, tau[w]))
        if len(tau_wm_m_filtered) == 1:
            # print(f"Found tuple: {tau_wm_m_filtered[0]}")
            tau_wm_m = tau_wm_m_filtered[0]

        tau_wm = tau_wm_m[0]
        if ft > tau_wm or tau_wm > lt:
            print(f"미사일 {m}, 발사대 {w}로 타깃 {t} (공격 시간 : {tau_wm})")
            print(f"최초 공격 가능 시점 : {ft}, 최후 발사 가능 시점 : {lt}")
            
            return False
    return True

"""
제약조건 (6)
각 발사대는 적재된 수량만큼 유도탄을 사용할 수 있으므로, 수식 (6)과 같이 한 발사대에서 할당할 수 있는 유도탄의 개수는 적재량 이하여야 한다는 제약 조건을 고려한다.
"""
def constraint6(theta_wt):
    W, T, M = theta_wt.shape
    for w in range(W):
        if np.sum(theta_wt[w,:,:]) > m_w[w]:
            return False
    return True

In [23]:
def simulation(tau, theta_wt):
    W, T, M = theta_wt.shape
    result = []
    for m in range(M):
        indices = np.where(theta_wt[:,:,m] == 1)
        if len(indices[0]) == 0 :
            continue
        w = int(indices[0])
        t = int(indices[1])
        
        ft = ft_wt[w][t]
        lt = lt_wt[w][t]

        tau_wm_m_filtered = list(filter(lambda x: x[1] == m, tau[w]))
        if len(tau_wm_m_filtered) == 1:
            # print(f"Found tuple: {tau_wm_m_filtered[0]}")
            tau_wm_m = tau_wm_m_filtered[0]

        tau_wm = tau_wm_m[0]
        PK = PK_wt[w][t]
        result.append((tau_wm, w, t, m, PK))

    if constraint2(theta_wt):
        print("제약조건 2 충족")
    if constraint3(theta_wt):
        print("제약조건 3 충족")
    if constraint4(tau):
        print("제약조건 4 충족")
    if constraint5(tau, theta_wt):
        print("제약조건 5 충족")
    if constraint6(theta_wt):
        print("제약조건 6 충족")

    result.sort()
    for res in result:

        print(f"{res[0]:.2f}초, 발사대 {res[1]}, 타깃 {res[2]}을 향해 미사일 {res[3]} 발사, 요격 확률 : {res[4]:.2f}")

    objective_value = objective_fun_1(theta_wt)

    print(f"목표함수 값 : {objective_value}")


In [24]:
simulation(tau, theta_wt)

제약조건 2 충족
제약조건 3 충족
제약조건 4 충족
제약조건 5 충족
제약조건 6 충족
1.45초, 발사대 4, 타깃 9을 향해 미사일 0 발사, 요격 확률 : 98.89
1.85초, 발사대 2, 타깃 2을 향해 미사일 2 발사, 요격 확률 : 94.65
2.18초, 발사대 0, 타깃 3을 향해 미사일 1 발사, 요격 확률 : 95.26
2.45초, 발사대 4, 타깃 7을 향해 미사일 3 발사, 요격 확률 : 91.03
2.47초, 발사대 3, 타깃 5을 향해 미사일 7 발사, 요격 확률 : 58.50
3.11초, 발사대 1, 타깃 8을 향해 미사일 4 발사, 요격 확률 : 87.29
3.18초, 발사대 0, 타깃 1을 향해 미사일 9 발사, 요격 확률 : 26.73
3.47초, 발사대 3, 타깃 4을 향해 미사일 8 발사, 요격 확률 : 56.43
3.91초, 발사대 2, 타깃 0을 향해 미사일 6 발사, 요격 확률 : 71.80
4.11초, 발사대 1, 타깃 6을 향해 미사일 5 발사, 요격 확률 : 87.23
목표함수 값 : 767.7984881239656
